In [1]:
# import the necessary packages
from tensorflow.keras.layers import BatchNormalization, Conv2D, Conv2DTranspose, LeakyReLU, Activation, Flatten, Dense, Input, MaxPool2D, MaxPooling2D, GlobalMaxPooling2D, Dropout, SeparableConv2D, Add, UpSampling2D
from tensorflow.keras.models import Model

from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import Sequential, losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from matplotlib import pyplot as plt

from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from mpl_toolkits.mplot3d import Axes3D
from sklearn_extra.cluster import KMedoids

from sklearn import metrics
# from google.colab.patches import cv2_imshow

import numpy as np
import os
# import cv2
import random
import pandas as pd
import json
import tensorflow as tf
import math
import time
import threading


gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

from tensorflow.keras.layers import Layer


2022-11-03 11:32:13.117352: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-03 11:32:13.297561: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-03 11:32:14.051569: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-11-03 11:32:14.051663: W tensorflow/stream_executor

In [5]:
# ssim = tf.image.ssim_multiscale(img1, img2, 1.0)

def load_image(path, im):
    image = (load_img(os.path.join(path, 
                                im), color_mode='grayscale'))
    image = img_to_array(image)
    # image = np.divide(image, 255)
    # image = image.astype('float32')
    return np.array(image)

def compute_ssim_matrix(img_list, path):
    ssim_mat = np.zeros((len(img_list), len(img_list)))

    for i in range(0, len(img_list)):
        print(img_list[i])
        im_1 = load_image(path, img_list[i])
        for j in range(0, len(img_list)):
            print(j)
            if i == j:
                ssim_mat[i, j] = 1
            else:
                im_2 = load_image(path, img_list[j])
                ssim_mat[i, j] = tf.image.ssim_multiscale(im_1, im_2, 1.0)
    
    return ssim_mat


gpus = tf.config.list_logical_devices('GPU')
with tf.device('/device:GPU:0'):
    img_list = sorted(os.listdir('/dev/shm/new_dataset_9000_11_08_grayscale_transparent/content/new_dataset_9000_11_08_grayscale_transparent/new_transparent/'))

    ssim_mat = compute_ssim_matrix(img_list, '/dev/shm/new_dataset_9000_11_08_grayscale_transparent/content/new_dataset_9000_11_08_grayscale_transparent/new_transparent/')


grayscale_transp_0001_0.png
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147


KeyboardInterrupt: 

In [4]:
print(np.zeros((len(img_list), len(img_list))).shape)

(29208, 29208)


In [3]:
gpus = tf.config.list_logical_devices('GPU')
print(gpus)

[LogicalDevice(name='/device:GPU:0', device_type='GPU')]


2022-11-02 09:35:08.246174: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-02 09:35:08.248274: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-02 09:35:08.251484: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-02 09:35:08.253982: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [ ]:
N = 2
M = 3
# elements = len(sorted(os.listdir('/content/new_dataset_9000_11_08/new_colored')))
elements = len(sorted(os.listdir('/dev/shm/new_dataset_9000_11_08_grayscale/content/new_dataset_9000_11_08_grayscale/new_colored')))
index_bound = math.floor(elements/N)

index_list = []

for i in range(N):
    start = i*index_bound
    stop = start + index_bound
    
    if i == (N-1):
        stop = elements

    index_list.append((start, stop))

print(index_list)

In [ ]:
"""
NEW
"""

batch_size = 64

# autoencoderBPR4 = AutoencoderBPR4()
# autoencoderBPR4.build([[batch_size, 224, 224, 1], [batch_size, 224, 224, 1]])
# autoencoderBPR4.load_weights('/content/autoencoderBPR4_19_09.hdf5')


import threading
     
shared_queue = []
max_ssim = 0
max_im = None

'''
the ImageReader should create several threads that make batches of images - with the size specified in batch_size parameter - and push them in a shared queue such that Inference
Threads can read and process them.
    '''
class ImageReader(threading.Thread):
    def __init__(self, name, img_dir, comparing_img_index, indexes, batch_size):
        threading.Thread.__init__(self)
        self.name = name
        self.img_dir = img_dir
        self.comparing_img_index = comparing_img_index
        self.indexes = indexes
        self.start_index = self.indexes[0]
        self.stop_index = self.indexes[1]
        self.batch_size = batch_size

    def run(self):

        print(f"Starting {self.name}")
        t1 = time.time()
        
        
        list_item = sorted(os.listdir(self.img_dir))[self.comparing_img_index]
        im_test = (img_to_array(load_img(os.path.join(self.img_dir, list_item), color_mode='grayscale'))/255).astype('float32')
        
        loop_list = sorted(os.listdir(self.img_dir))[self.start_index:self.stop_index]
        initial = len(loop_list)
        
        if list_item in loop_list:
            loop_list.remove(list_item) 
            
        dataloader_train_bpr = dataLoaderImagesAutoencoderBPR(self.img_dir, loop_list, self.img_dir, loop_list, self.batch_size)
        steps_per_epoch = len(loop_list) // self.batch_size 
        
        for i in range(steps_per_epoch):
            t2 = time.time()
            batches, _ = dataloader_train_bpr.__next__()
            global shared_queue
            shared_queue.append(batches)
            t3 = time.time()
            
            print(f"{self.name} added one batch of {self.batch_size} in the shared queue in {t3-t2} sec.")
        
        t4 = time.time()
        
        print(f"{self.name} has finished in {t4-t1}")
        
        
    
'''
AutoencoderInference threads take batches of images from the shared queue, make inference of them and remove the elements in queue.
'''
class AutoencoderInference (threading.Thread):
    def __init__(self, name, img_dir, comparing_img_index, batch_size, autoencoder):
        threading.Thread.__init__(self)
        self.name = name
        self.img_dir = img_dir
        self.comparing_img_index = comparing_img_index
        self.batch_size = batch_size
        self.autoencoder = autoencoder
        
    
    def run(self):
        print(f"Starting {self.name}")
        t1 = time.time()
        
        list_item = sorted(os.listdir(self.img_dir))[self.comparing_img_index]
        im_test = (img_to_array(load_img(os.path.join(self.img_dir, list_item), color_mode='grayscale'))/255).astype('float32')
        
        # imgs = shared_queue[:self.batch_size]
        global shared_queue
        global readers
        
        
        while all([r.is_alive() for r in readers]):
            if len(shared_queue) == 0:
                time.sleep(0.01)
                continue
            
            imgs = shared_queue[:1]
            imgs = imgs[0]
            # print(f"{self.name} has read imgs of shape {len(imgs)}{imgs[0].shape}")
            # shared_queue = shared_queue[self.batch_size:]
            shared_queue = shared_queue[1:]
            # shared_queue.remove(imgs)

            t2 = time.time()
            _, ssimi = self.autoencoder.predict([imgs, np.expand_dims(im_test, 0)])
            t3 = time.time()

            # print(f"ssimi shape {ssimi[0].shape}")


            print(f"{self.name} has done inference in {t3 - t2} sec.")

            for i, s in enumerate(ssimi):
                global max_ssim
                # print(f"type of s {type(s)}, {s}")
                if s > max_ssim:
                    max_ssim = s
                    global max_im
                    # print(f"imgs is {imgs}")
                    print(f"{self.name} imgs len {len(imgs)}, i is {i}")
                    max_im = imgs[i]

            t4 = time.time()
        
        print(f"{self.name} has finished in {t4 - t1} sec.")
        
                                            
readers, inferencers = [], []
for t in range(N):
    readers.append(ImageReader("reader_{}".format(t+1), 
                               '/dev/shm/new_dataset_9000_11_08_grayscale/content/new_dataset_9000_11_08_grayscale/new_colored', 
                                4560, 
                                index_list[t],
                                batch_size
                              ))

for s in range(M):
    
    inferencers.append(AutoencoderInference("inference_{}".format(s+1), '/dev/shm/new_dataset_9000_11_08_grayscale/content/new_dataset_9000_11_08_grayscale/new_colored', 
                        4560, 
                        64,
                        autoencoderBPR4)
                      )
    

for t in readers:
    t.start()
    

# time.sleep(1)
# print(f"len of queue {len(shared_queue)}")
                                            
for t in inferencers:
    t.start()
    
for t in readers:    
    t.join()
                                            
for t in inferencers:    
    t.join()
            
in_im = (img_to_array(load_img(os.path.join(inferencers[0].img_dir, sorted(os.listdir(inferencers[0].img_dir))[inferencers[0].comparing_img_index]), color_mode='grayscale')))
plt.imshow(in_im)
plt.show()
plt.imshow((max_im*255).astype('uint8'))
plt.show()

In [24]:
# ssim_mat = np.zeros((len(img_list), len(img_list)))
img_list = sorted(os.listdir('/dev/shm/new_dataset_9000_11_08_grayscale_transparent/content/new_dataset_9000_11_08_grayscale_transparent/new_transparent/'))
path = '/dev/shm/new_dataset_9000_11_08_grayscale_transparent/content/new_dataset_9000_11_08_grayscale_transparent/new_transparent/'

img_list = img_list[:100]
ssim_mat = np.zeros((len(img_list), len(img_list)))

def load_image(path, im):
    image = (load_img(os.path.join(path, 
                                im), color_mode='grayscale'))
    image = img_to_array(image)
    # image = np.divide(image, 255)
    # image = image.astype('float32')
    return np.array(image)

def compute_ssim_matrix(img_list, path, start_index, stop_index):
    # ssim_mat = np.zeros((len(img_list), len(img_list)))
    global ssim_mat

    for i in range(start_index, stop_index):
        # print(img_list[i])
        im_1 = load_image(path, img_list[i])
        # print(im_1.shape)
        for j in range(0, len(img_list)):
            # print(j)
            if i == j:
                ssim_mat[i, j] = 1
            else:
                im_2 = load_image(path, img_list[j])
                # print(im_2.shape)
                with tf.device('/device:GPU:0'):
                    ssim_mat[i, j] = np.array(tf.image.ssim_multiscale(np.expand_dims(im_1, 0), np.expand_dims(im_2, 0), 255.0))


class MatrixBuilder(threading.Thread):
    def __init__(self, name, img_dir, img_list, indexes):
        threading.Thread.__init__(self)
        self.name = name
        self.img_dir = img_dir
        self.img_list = img_list
        self.start_index = indexes[0]
        self.stop_index = indexes[1]

    def run(self):

        print(f"Starting {self.name}")
        t1 = time.time()

        compute_ssim_matrix(self.img_list, self.img_dir, self.start_index, self.stop_index)
        
        t2 = time.time()
        
        print(f"{self.name} has finished in {t2-t1}")


N = 10

index_bound = math.floor(len(img_list)/N)

index_list = []

for i in range(N):
    start = i*index_bound
    stop = start + index_bound
    
    if i == (N-1):
        stop = len(img_list)

    index_list.append((start, stop))


matrix_builders = []
for t in range(N):
    matrix_builders.append(MatrixBuilder(f"builder_{t+1}", path, img_list, index_list[t]))

for t in matrix_builders:
    t.start()
    
for t in matrix_builders:    
    t.join()

Starting builder_1
Starting builder_2
Starting builder_3
Starting builder_4
Starting builder_5
Starting builder_6
Starting builder_8
Starting builder_7
Starting builder_9
Starting builder_10


Exception in thread builder_9:
Traceback (most recent call last):
  File "/opt/conda/envs/matteovenv/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/tmp/ipykernel_7774/1131430940.py", line 49, in run
  File "/tmp/ipykernel_7774/1131430940.py", line 32, in compute_ssim_matrix
  File "/opt/conda/envs/matteovenv/lib/python3.9/site-packages/tensorflow/python/util/traceback_utils.py", line 153, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/opt/conda/envs/matteovenv/lib/python3.9/site-packages/tensorflow/python/ops/control_flow_ops.py", line 156, in Assert
    raise errors.InvalidArgumentError(
tensorflow.python.framework.errors_impl.InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: 1, 112, 112, 1
11
Exception in thread builder_3:
Traceback (most recent call last):
  File "/opt/conda/envs/matteovenv/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  Fil

builder_5 has finished in 122.28273129463196


In [23]:
print(ssim_mat)
# with open('ssim_mat.npy', 'wb') as f:
#     np.save(f, ssim_mat)

ssim_mat_def = ssim_mat

[[1.         0.         0.         ... 0.11152816 0.09128559 0.0876879 ]
 [0.         1.         0.         ... 0.15490602 0.10704208 0.23935917]
 [0.         0.         1.         ... 0.         0.22359428 0.        ]
 ...
 [0.11152816 0.15490602 0.         ... 1.         0.18416576 0.24589823]
 [0.09128559 0.10704208 0.22359428 ... 0.18416576 1.         0.26063752]
 [0.0876879  0.23935917 0.         ... 0.24589823 0.26063752 1.        ]]


In [5]:
path = '/dev/shm/new_dataset_9000_11_08_grayscale_transparent/content/new_dataset_9000_11_08_grayscale_transparent/new_transparent/'

def load_image(path, im):
    im_list = []
    if len(im) == 1:
        image = (load_img(os.path.join(path, 
                                    im), color_mode='grayscale'))
        image = img_to_array(image)
        return np.array(image)

    for i in im:
        # print(i)
        image = (load_img(os.path.join(path, 
                                    i), color_mode='grayscale'))
        image = img_to_array(image)
        # image = np.divide(image, 255)
        # image = image.astype('float32')
        im_list.append(image)
    return np.array(im_list)

def load_single_image(path, im):
    image = (load_img(os.path.join(path, 
                                    im), color_mode='grayscale'))
    image = img_to_array(image)
    return np.array(image)

img_list = sorted(os.listdir('/dev/shm/new_dataset_9000_11_08_grayscale_transparent/content/new_dataset_9000_11_08_grayscale_transparent/new_transparent/'))

# aa = load_image('/dev/shm/new_dataset_9000_11_08_grayscale_transparent/content/new_dataset_9000_11_08_grayscale_transparent/new_transparent/', img_list[0])
bb = load_image('/dev/shm/new_dataset_9000_11_08_grayscale_transparent/content/new_dataset_9000_11_08_grayscale_transparent/new_transparent/', img_list[0:100])
print(len(np.array(tf.image.ssim_multiscale(load_single_image(path, img_list[0]), bb, 1.0))))
print(np.array(tf.image.ssim_multiscale(load_single_image(path, img_list[0]), bb, 255.0)))

100
[1.         0.         0.         0.14769946 0.19159353 0.22834542
 0.07992305 0.06644374 0.         0.09343289 0.17464422 0.23407428
 0.         0.13412827 0.12083033 0.19385885 0.         0.12655601
 0.20315586 0.         0.16770999 0.         0.12738536 0.0676963
 0.         0.2235302  0.         0.17469095 0.         0.
 0.15212394 0.         0.         0.14216502 0.19649768 0.
 0.0985119  0.         0.         0.         0.20405896 0.23195818
 0.09244111 0.2142893  0.         0.22987732 0.         0.
 0.10347489 0.0924172  0.         0.         0.13120624 0.
 0.16268696 0.23161855 0.         0.         0.09678528 0.1218698
 0.         0.14791475 0.09571249 0.1942762  0.13676122 0.09930456
 0.         0.         0.         0.         0.         0.21756254
 0.         0.06917217 0.12118762 0.         0.11011674 0.08887351
 0.09055348 0.12084308 0.14028193 0.1833994  0.         0.24142571
 0.         0.09578396 0.         0.         0.         0.
 0.1303015  0.         0.10751683

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

path = '/dev/shm/new_dataset_9000_11_08_grayscale_transparent/content/new_dataset_9000_11_08_grayscale_transparent/new_transparent/'
img_list = sorted(os.listdir('/dev/shm/new_dataset_9000_11_08_grayscale_transparent/content/new_dataset_9000_11_08_grayscale_transparent/new_transparent/'))

def load_image(path, im):
    im_list = []
    if len(im) == 1:
        image = (load_img(os.path.join(path, 
                                    im), color_mode='grayscale'))
        image = img_to_array(image)
        return np.array(image)

    for i in im:
        # print(i)
        image = (load_img(os.path.join(path, 
                                    i), color_mode='grayscale'))
        image = img_to_array(image)
        # image = np.divide(image, 255)
        # image = image.astype('float32')
        im_list.append(image)
    return np.array(im_list)

def load_single_image(path, im):
    image = (load_img(os.path.join(path, 
                                    im), color_mode='grayscale'))
    image = img_to_array(image)
    return np.array(image)


def evaluate_ssim_in_batch(path, img_list):
    ssim_mat = []
    for i, im in enumerate(img_list):
        if i==0:
            im = load_image(path, img_list)
        ssim_mat.append(tf.image.ssim_multiscale(load_single_image(path, img_list[i]), im, 255.0))
        print(i)
    return np.array(ssim_mat)

ssim_mat = evaluate_ssim_in_batch(path, img_list)
print(ssim_mat.shape)
print(ssim_mat)

2022-11-03 08:23:53.186058: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-03 08:23:53.188528: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 08:23:53.191835: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 08:23:53.194537: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

ResourceExhaustedError: {{function_node __wrapped__DepthwiseConv2dNative_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[29208,1,214,214] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:DepthwiseConv2dNative]

03/11/2022

In [2]:
# np.concatenate(index_list).ravel()

def evaluate_ssim_in_batch(path, img_list):
    ssim_mat = []
    img = load_image(path, img_list)
    print('CARICATO')
    for i, im in enumerate(img_list):
        # if i==0:
        #     im = load_image(path, img_list)
        ssim = tf.image.ssim_multiscale(img[i], img, 255.0)
        ssim_mat.append(ssim)
        del ssim
        print(f"iterazione {i} con calcolo ssim")
    return np.array(ssim_mat)


def evaluate_ssim_batch_len(img_list, ssim_mat_batch):
    ssim_mat_bound = math.floor(len(img_list)/ssim_mat_batch)

    ssim_index_list = []

    for i in range(ssim_mat_batch):
        start = i*ssim_mat_bound
        stop = start + ssim_mat_bound
        
        if i == (ssim_mat_batch-1):
            stop = len(img_list)

        ssim_index_list.append((start, stop))

    return ssim_index_list

def load_image(path, im):
    im_list = []
    if len(im) == 1:
        image = (load_img(os.path.join(path, 
                                    im), color_mode='grayscale'))
        image = img_to_array(image)
        return np.array(image)

    for i in im:
        # print(i)
        image = (load_img(os.path.join(path, 
                                    i), color_mode='grayscale'))
        image = img_to_array(image)
        # image = np.divide(image, 255)
        # image = image.astype('float32')
        im_list.append(image)
    return np.array(im_list)

def load_single_image(path, im):
    image = (load_img(os.path.join(path, 
                                    im), color_mode='grayscale'))
    image = img_to_array(image)
    return np.array(image)


def evaluate_ssim_in_batch_2(path, img_list, ssim_mat_batch):
    ssim_mat = []
    ssim_index_list = evaluate_ssim_batch_len(img_list, ssim_mat_batch)

    for i, im in enumerate(img_list):
        batch_ssim_list = []
        for batch in ssim_index_list:
            start, stop = batch
            img_list_batch = load_image(path, img_list[start:stop])
            batch_ssim_list.append(tf.image.ssim_multiscale(load_single_image(path, img_list[i]), img_list_batch, 255.0))
            del img_list_batch
        
        ssim_mat.append(np.concatenate(batch_ssim_list).ravel())
        print(i)

    return np.array(ssim_mat)



path = '/dev/shm/new_dataset_9000_11_08_grayscale_transparent/content/new_dataset_9000_11_08_grayscale_transparent/new_transparent/'
img_list = sorted(os.listdir('/dev/shm/new_dataset_9000_11_08_grayscale_transparent/content/new_dataset_9000_11_08_grayscale_transparent/new_transparent/'))
ssim_mat = evaluate_ssim_in_batch_2(path, img_list, 10)
# ssim_mat = evaluate_ssim_in_batch(path, img_list)

2022-11-03 11:06:39.312172: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-03 11:06:39.314486: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 11:06:39.317891: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 11:06:39.320517: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36


KeyboardInterrupt: 

In [2]:
def evaluate_ssim_batch_len(img_list, ssim_mat_batch):
    ssim_mat_bound = math.floor(len(img_list)/ssim_mat_batch)

    ssim_index_list = []

    for i in range(ssim_mat_batch):
        start = i*ssim_mat_bound
        stop = start + ssim_mat_bound
        
        if i == (ssim_mat_batch-1):
            stop = len(img_list)

        ssim_index_list.append((start, stop))

    return ssim_index_list

def load_image(path, im):
    im_list = []
    if len(im) == 1:
        image = (load_img(os.path.join(path, 
                                    im), color_mode='grayscale'))
        image = img_to_array(image)
        return np.array(image)

    for i in im:
        # print(i)
        image = (load_img(os.path.join(path, 
                                    i), color_mode='grayscale'))
        image = img_to_array(image)
        # image = np.divide(image, 255)
        # image = image.astype('float32')
        im_list.append(image)
    return np.array(im_list)

def load_single_image(path, im):
    image = (load_img(os.path.join(path, 
                                    im), color_mode='grayscale'))
    image = img_to_array(image)
    return np.array(image)

def evaluate_ssim_in_batch_3(path, img_list, ssim_mat_batch, indexes):
    ssim_mat = []
    ssim_index_list = evaluate_ssim_batch_len(img_list, ssim_mat_batch)
    start, stop = indexes

    for i, im in enumerate(img_list[start:stop]):
        batch_ssim_list = []
        for batch in ssim_index_list:
            start, stop = batch
            img_list_batch = load_image(path, img_list[start:stop])
            batch_ssim_list.append(tf.image.ssim_multiscale(load_single_image(path, img_list[i]), img_list_batch, 255.0))
            del img_list_batch
        
        ssim_mat.append(np.concatenate(batch_ssim_list).ravel())
        print(i)

    return np.array(ssim_mat)


class MatrixBuilder(threading.Thread):
    def __init__(self, name, img_dir, img_list, ssim_mat_batch, indexes):
        threading.Thread.__init__(self)
        self.name = name
        self.img_dir = img_dir
        self.img_list = img_list
        self.ssim_mat_batch = ssim_mat_batch
        self.indexes = indexes
        self.start_index = indexes[0]
        self.stop_index = indexes[1]

    def run(self):

        print(f"Starting {self.name}")
        t1 = time.time()

        evaluate_ssim_in_batch_3(self.img_dir, self.img_list, self.ssim_mat_batch, self.indexes)
        
        t2 = time.time()
        
        print(f"{self.name} has finished in {t2-t1}")



N = 10
path = '/dev/shm/new_dataset_9000_11_08_grayscale_transparent/content/new_dataset_9000_11_08_grayscale_transparent/new_transparent/'
img_list = sorted(os.listdir('/dev/shm/new_dataset_9000_11_08_grayscale_transparent/content/new_dataset_9000_11_08_grayscale_transparent/new_transparent/'))
index_list = evaluate_ssim_batch_len(img_list, N)
matrix_builders = []

print(index_list[0])

for t in range(N):
    matrix_builders.append(MatrixBuilder(f"builder_{t+1}", path, img_list, 64, index_list[t]))

for t in matrix_builders:
    t.start()
    
for t in matrix_builders:    
    t.join()

(0, 2920)
Starting builder_1
Starting builder_2
Starting builder_3
Starting builder_4
Starting builder_5
Starting builder_6
Starting builder_7
Starting builder_8
Starting builder_9
Starting builder_10


2022-11-03 11:32:22.462058: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-03 11:32:22.464111: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 11:32:22.467400: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-03 11:32:22.470004: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


KeyboardInterrupt: 